# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He, load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_DFT(lattice, atoms, positions;
                  functionals=LDA(), symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770466669193                   -0.53    9.0    193ms
  2   -2.771690261241       -2.91       -1.32    1.0    133ms
  3   -2.771714448227       -4.62       -2.51    1.0    445ms
  4   -2.771714660150       -6.67       -3.29    1.0    111ms
  5   -2.771714711727       -7.29       -3.76    2.0    143ms
  6   -2.771714715120       -8.47       -4.76    1.0    112ms
  7   -2.771714715248       -9.89       -5.29    2.0    111ms
  8   -2.771714715249      -11.76       -6.02    1.0    102ms
  9   -2.771714715250      -12.41       -6.60    2.0    115ms
 10   -2.771714715250      -14.05       -7.09    1.0    101ms
 11   -2.771714715250   +  -14.24       -7.65    1.0    104ms
 12   -2.771714715250   +  -14.21       -8.10    1.0    104ms


-0.00013457304697586344

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_DFT(lattice, atoms, positions;
                    functionals=LDA(),
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770475947427                   -0.52    9.0    217ms
  2   -2.771777670071       -2.89       -1.32    1.0    144ms
  3   -2.771801608736       -4.62       -2.45    1.0    151ms
  4   -2.771801990290       -6.42       -3.15    1.0    436ms
  5   -2.771802073833       -7.08       -4.04    2.0    108ms
  6   -2.771802074111       -9.56       -4.18    1.0    102ms
  7   -2.771802074465       -9.45       -5.03    1.0    105ms
  8   -2.771802074475      -11.03       -5.41    1.0    102ms
  9   -2.771802074476      -11.88       -6.58    1.0    113ms
 10   -2.771802074476      -13.32       -6.44    2.0    116ms
 11   -2.771802074476      -13.55       -7.51    1.0    105ms
 12   -2.771802074476   +  -14.88       -7.70    1.0    105ms
 13   -2.771802074476      -14.12       -8.00    1.0    106ms


0.01761222206571159

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457304697586344
Displaced dipole:  0.01761222206571159
Polarizability :   1.7746795112687452


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response (also known as density-functional perturbation theory)
to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920785154e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766551396997e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852765369991e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694603397963e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088789138733e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.295433338168e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 1.299921455383e-12
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 1.299921455383e-12
[ Info: GMRES linsolve in iter 2; step 1: normres = 9.770705105057e-11
[ Info: GMRES linsolve in iter 2; step 2: normres = 1.565879634084e-11
[ Info: GMRES linsolve in iter 2; step 3: normres = 3.266127297488e-13
[ Info: GMRES linsolve in iter 2; finished at step 3: normres = 3.266127297488e-13
┌ Info: GMRES linsolve converged at iteration 2, step 3:
│ *  norm of residual = 3.265835706661194e-13
└ *  number of operat

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.